# Many to One Explainer - Demo of Conjunctive vs Disjunctive Explanations

This demo measures the differences in quality metrics and runtime of explanation generation between conjunctive and disjunctive explanations, on various datasets.\
Each dataset has multiple tests, and at the bottom of each section, the average results are displayed.\
Additionally, at the bottom of the notebook, a collection of all the average results + average across all datasets is displayed.

## Setup

In [1]:
import pandas as pd
import pd_explain
import time
import numpy as np
from sklearn.cluster import KMeans

usetex-False
(CVXPY) Jan 24 11:30:57 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jan 24 11:30:57 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')


For this demo, we use the default separation error threshold of 0.3, to represent a real world scenario.\
However, if you would like to change it, you can do so by setting the `separation_error` variable below.

In [3]:
separation_error = 0.3

By default, we use sampling to speed up the explanation generation process.

In [4]:
pd_explain.get_use_sampling_value()

True

This can be changed either per function call or globally.\
If you would like to run this demo without sampling, uncomment and run the cell below prior to running the rest of the notebook.

In [5]:
# pd_explain.toggle_sampling(False)
# pd_explain.get_use_sampling_value()

In [6]:
def build_comparison_df(comparison_df, test_name, explanation_type, res_df, runtime):
    try:
        # Each group without an explanation has 1 row with "No explanation found" in the 'Explanation' column / index
        num_groups_not_found = res_df.loc[:, 'No explanation found', :].shape[0]
        # Each group that does have explanations can have multiple rows, and we want to count the number of groups, which is the length of the index on the first level
        num_groups_total = len(res_df.index.levels[0])
        percentage_not_found = (num_groups_not_found / num_groups_total) * 100
    # This should only happen in case explanations were found for all instances
    except KeyError:
        percentage_not_found = 0
    try:
        # Get the string representation of the explanations, for each explanation that is not "No explanation found"
        explanations = res_df.index.get_level_values(1)[res_df.index.get_level_values(1) != 'No explanation found'].to_series()
        # Measure the length of each explanation, by counting the number of 'AND' or 'OR' substrings
        explanation_lengths = explanations.apply(lambda x: x.count('AND') + x.count('OR') + 1)
        # Calculate the average explanation length
        avg_explanation_length = explanation_lengths.mean()
        num_explanations_found = len(explanation_lengths)
    # This should only happen in case explanations were not found for any instance
    except (ZeroDivisionError, KeyError):
        avg_explanation_length = pd.NA
        num_explanations_found = 0

    comparison_df.loc[(test_name, explanation_type), :] = [np.mean(res_df['Coverage']), np.mean(res_df['Separation Error']), percentage_not_found, avg_explanation_length, num_explanations_found, runtime]

In [7]:
columns = ['Average Coverage', 'Average Separation Error', '% Explanation Not Found', 'Average Explanation Length', 'Number of Explanations Found', 'Runtime']
index = pd.MultiIndex(levels=[[], []], codes=[[], []], names=['Test', 'Explanation Type'])

## Adults dataset

In [8]:
adults = pd.read_csv(r"..\Examples\Datasets\adult.csv")
adults

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [9]:
adults_comparison_df = pd.DataFrame(columns=columns, index=index)

### Full dataset test

In [10]:
start_time = time.time()
conj_res = adults.explain(explainer='many to one', labels='label', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, 'Full dataset', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.28957557678222656


Coverage  \
Group / Cluster Explanation                                                   
<=50K           0 <= capital-gain <= 5095.5 AND 1 <= education-...     0.75   
                0 <= capital-gain <= 4243.5                            0.99   
                1 <= education-num <= 10                               0.75   
                0 <= capital-gain <= 5095.5                             1.0   
>50K            No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
<=50K           0 <= capital-gain <= 5095.5 AND 1 <= education-...             0.13  
                0 <= capital-gain <= 4243.5                                     0.2  
                1 <= education-num <= 10                                       0.15  
                0 <= capital-gain <= 5095.5                                    0.21  
>50K            No explanation found                                            NaN

In [11]:
start_time = time.time()
disj_res = adults.explain(explainer='many to one', labels='label', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, 'Full dataset', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.34167981147766113


Coverage Separation Error
Group / Cluster Explanation                                          
<=50K           1 <= education-num <= 10        0.75             0.15
                0 <= capital-gain <= 4243.5     0.99              0.2
                0 <= capital-gain <= 5095.5      1.0             0.21
>50K            No explanation found             NaN              NaN

### Numeric only and categorical only tests

In [12]:
numeric_only = adults[['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']]
categorical_only = adults[['workclass', 'education', 'marital-status', 'occupation', 'relationship']]
labels = adults['label']

In [13]:
start_time = time.time()
conj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, 'Numeric only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.8245134353637695


Coverage  \
Group / Cluster Explanation                                                   
<=50K           0 <= capital-gain <= 5095.5 AND 1 <= education-...     0.75   
                0 <= capital-gain <= 4243.5                            0.99   
                1 <= education-num <= 10                               0.75   
                0 <= capital-gain <= 5095.5                             1.0   
>50K            No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
<=50K           0 <= capital-gain <= 5095.5 AND 1 <= education-...             0.13  
                0 <= capital-gain <= 4243.5                                     0.2  
                1 <= education-num <= 10                                       0.15  
                0 <= capital-gain <= 5095.5                                    0.21  
>50K            No explanation found                                            NaN

In [14]:
start_time = time.time()
disj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, 'Numeric only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.6171953678131104


Coverage Separation Error
Group / Cluster Explanation                                          
<=50K           1 <= education-num <= 10        0.75             0.15
                0 <= capital-gain <= 4243.5     0.99              0.2
                0 <= capital-gain <= 5095.5      1.0             0.21
>50K            No explanation found             NaN              NaN

In [15]:
start_time = time.time()
conj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, 'Categorical only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.09193849563598633


Coverage  \
Group / Cluster Explanation                                                   
<=50K           occupation != Prof-specialty AND occupation != ...     0.82   
                occupation != Exec-managerial                          0.91   
                occupation != Prof-specialty                           0.91   
>50K            No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
<=50K           occupation != Prof-specialty AND occupation != ...             0.17  
                occupation != Exec-managerial                                  0.21  
                occupation != Prof-specialty                                   0.21  
>50K            No explanation found                                            NaN

In [16]:
start_time = time.time()
disj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, 'Categorical only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.11255860328674316


Coverage  \
Group / Cluster Explanation                                                   
<=50K           occupation != Exec-managerial                          0.91   
                occupation != Prof-specialty                           0.91   
                occupation != Prof-specialty OR occupation != E...      1.0   
>50K            No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
<=50K           occupation != Exec-managerial                                  0.21  
                occupation != Prof-specialty                                   0.21  
                occupation != Prof-specialty OR occupation != E...             0.24  
>50K            No explanation found                                            NaN

### Groupby tests

In [17]:
single_attribute_groupby = adults.groupby('relationship').mean()
multi_attribute_groupby = adults.groupby(['sex', 'relationship']).mean()

In [18]:
start_time = time.time()
conj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, 'Single attribute groupby', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 1.3326494693756104


Coverage  \
Group / Cluster Explanation                                                   
Husband         marital-status == Married-civ-spouse AND sex !=...      1.0   
                marital-status == Married-civ-spouse                    1.0   
Wife            marital-status == Married-civ-spouse AND sex ==...     0.99   
Not-in-family   No explanation found                                    NaN   
Other-relative  No explanation found                                    NaN   
Own-child       No explanation found                                    NaN   
Unmarried       No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
Husband         marital-status == Married-civ-spouse AND sex !=...             0.01  
                marital-status == Married-civ-spouse                           0.11  
Wife            marital-status == Married-civ-spouse AND sex ==...             0.05  
Not-in-family   No explanation found                                            NaN  
Other-relative  No explanation found                                            NaN  
Own-child       No explanation found                                            NaN  
Unmarried       No explanation found                                            NaN

In [19]:
start_time = time.time()
disj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, 'Single attribute groupby', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 1.4520368576049805


,,Coverage,Separation Error
Group / Cluster,Explanation,,
Husband,marital-status == Married-civ-spouse,1.0,0.11
Not-in-family,No explanation found,NaN,NaN
Other-relative,No explanation found,NaN,NaN
Own-child,No explanation found,NaN,NaN
Unmarried,No explanation found,NaN,NaN
Wife,No explanation found,NaN,NaN


In [20]:
start_time = time.time()
conj_res = multi_attribute_groupby.explain(explainer='many to one', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, 'Multi attribute groupby', 'conjunctive', conj_res, conj_time)
conj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Conjunctive explanation runtime: 2.692655086517334


Coverage  \
Group / Cluster              Explanation                                                   
('Male', 'Husband')          marital-status == Married-civ-spouse                    1.0   
                             marital-status == Married-civ-spouse AND occupa...     0.95   
                             marital-status == Married-civ-spouse AND 39.5 <...     0.88   
                             marital-status == Married-civ-spouse AND 38.5 <...     0.88   
                             marital-status == Married-civ-spouse AND 40 <= ...     0.88   
('Female', 'Not-in-family')  No explanation found                                    NaN   
('Female', 'Other-relative') No explanation found                                    NaN   
('Female', 'Own-child')      No explanation found                                    NaN   
('Female', 'Unmarried')      No explanation found                                    NaN   
('Female', 'Wife')           No explanation found                                    NaN   
('Male', 'Not-in-family')    No explanation found                                    NaN   
('Male', 'Other-relative')   No explanation found                                    NaN   
('Male', 'Own-child')        No explanation found                                    NaN   
('Male', 'Unmarried')        No explanation found                                    NaN   

                                                                                Separation Error  
Group / Cluster              Explanation                                                          
('Male', 'Husband')          marital-status == Married-civ-spouse                           0.11  
                             marital-status == Married-civ-spouse AND occupa...             0.09  
                             marital-status == Married-civ-spouse AND 39.5 <...             0.08  
                             marital-status == Married-civ-spouse AND 38.5 <...             0.08  
                             marital-status == Married-civ-spouse AND 40 <= ...             0.08  
('Female', 'Not-in-family')  No explanation found                                            NaN  
('Female', 'Other-relative') No explanation found                                            NaN  
('Female', 'Own-child')      No explanation found                                            NaN  
('Female', 'Unmarried')      No explanation found                                            NaN  
('Female', 'Wife')           No explanation found                                            NaN  
('Male', 'Not-in-family')    No explanation found                                            NaN  
('Male', 'Other-relative')   No explanation found                                            NaN  
('Male', 'Own-child')        No explanation found                                            NaN  
('Male', 'Unmarried')        No explanation found                                            NaN

In [21]:
start_time = time.time()
disj_res = multi_attribute_groupby.explain(explainer='many to one', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, 'Multi attribute groupby', 'disjunctive', disj_res, disj_time)
disj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Disjunctive explanation runtime: 2.7848727703094482


,,Coverage,Separation Error
Group / Cluster,Explanation,,
"('Male', 'Husband')",marital-status == Married-civ-spouse,1.0,0.11
"('Female', 'Not-in-family')",No explanation found,NaN,NaN
"('Female', 'Other-relative')",No explanation found,NaN,NaN
"('Female', 'Own-child')",No explanation found,NaN,NaN
"('Female', 'Unmarried')",No explanation found,NaN,NaN
"('Female', 'Wife')",No explanation found,NaN,NaN
"('Male', 'Not-in-family')",No explanation found,NaN,NaN
"('Male', 'Other-relative')",No explanation found,NaN,NaN
"('Male', 'Own-child')",No explanation found,NaN,NaN


### Clustering test

#### 3 clusters

In [22]:
cluster_labels = KMeans(n_clusters=3).fit_predict(pd.get_dummies(adults))

C:\Users\Yuval\PycharmProjects\FEDEx_Generator\.venv1\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\Yuval\PycharmProjects\FEDEx_Generator\.venv1\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\Yuval\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Yuval\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  Fi

In [23]:
start_time = time.time()
conj_res = adults.explain(explainer='many to one', labels=cluster_labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, '3 clusters', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.8298277854919434


,,Coverage,Separation Error
Group / Cluster,Explanation,,
0,291675.5 <= fnlwgt <= 1490400,1.0,0.0
1,13769 <= fnlwgt <= 149628.5,1.0,0.0
2,149628.5 <= fnlwgt <= 291675.5,1.0,0.0


In [24]:
start_time = time.time()
disj_res = adults.explain(explainer='many to one', labels=cluster_labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, '3 clusters', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.9882702827453613


,,Coverage,Separation Error
Group / Cluster,Explanation,,
0,291675.5 <= fnlwgt <= 1490400,1.0,0.0
1,13769 <= fnlwgt <= 149628.5,1.0,0.0
2,149628.5 <= fnlwgt <= 291675.5,1.0,0.0


#### 15 clusters

In [25]:
cluster_labels = KMeans(n_clusters=15).fit_predict(pd.get_dummies(adults))

In [26]:
start_time = time.time()
conj_res = adults.explain(explainer='many to one', labels=cluster_labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, '15 clusters', 'conjunctive', conj_res, conj_time)
conj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Conjunctive explanation runtime: 4.713717460632324


Coverage Separation Error
Group / Cluster Explanation                                             
0               145798.0 <= fnlwgt <= 179933.0      1.0              0.0
2               68158.5 <= fnlwgt <= 110200.5       1.0              0.0
5               19914 <= fnlwgt <= 68158.5          1.0              0.0
6               405736.0 <= fnlwgt <= 488706        1.0              0.0
9               110200.5 <= fnlwgt <= 145798.0      1.0              0.0
11              176467 <= fnlwgt <= 207202         0.84              0.1
                176467 <= fnlwgt <= 231413          1.0             0.28
1               No explanation found                NaN              NaN
3               No explanation found                NaN              NaN
7               No explanation found                NaN              NaN
12              No explanation found                NaN              NaN

In [27]:
start_time = time.time()
disj_res = adults.explain(explainer='many to one', labels=cluster_labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, '15 clusters', 'disjunctive', disj_res, disj_time)
disj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Disjunctive explanation runtime: 4.624136686325073


Coverage Separation Error
Group / Cluster Explanation                                             
0               145798.0 <= fnlwgt <= 179933.0      1.0              0.0
2               68158.5 <= fnlwgt <= 110200.5       1.0              0.0
5               19914 <= fnlwgt <= 68158.5          1.0              0.0
6               405736.0 <= fnlwgt <= 488706        1.0              0.0
9               110200.5 <= fnlwgt <= 145798.0      1.0              0.0
11              176467 <= fnlwgt <= 207202         0.84              0.1
                176467 <= fnlwgt <= 231413          1.0             0.28
1               No explanation found                NaN              NaN
3               No explanation found                NaN              NaN
7               No explanation found                NaN              NaN
12              No explanation found                NaN              NaN

### Summary of results

In [28]:
conj_averages = adults_comparison_df.loc[:, 'conjunctive', :].mean()
disj_averages = adults_comparison_df.loc[:, 'disjunctive', :].mean()
adults_comparison_df.loc[('Average', 'conjunctive'), :] = conj_averages
adults_comparison_df.loc[('Average', 'disjunctive'), :] = disj_averages

In [29]:
adults_comparison_df

Average Coverage  \
Test                     Explanation Type                    
Full dataset             conjunctive                0.8725   
                         disjunctive              0.913333   
Numeric only             conjunctive                0.8725   
                         disjunctive              0.913333   
Categorical only         conjunctive                  0.88   
                         disjunctive                  0.94   
Single attribute groupby conjunctive              0.996667   
                         disjunctive                   1.0   
Multi attribute groupby  conjunctive                 0.918   
                         disjunctive                   1.0   
3 clusters               conjunctive                   1.0   
                         disjunctive                   1.0   
15 clusters              conjunctive              0.977143   
                         disjunctive              0.977143   
Average                  conjunctive              0.930973   
                         disjunctive              0.963401   

                                          Average Separation Error  \
Test                     Explanation Type                            
Full dataset             conjunctive                        0.1725   
                         disjunctive                      0.186667   
Numeric only             conjunctive                        0.1725   
                         disjunctive                      0.186667   
Categorical only         conjunctive                      0.196667   
                         disjunctive                          0.22   
Single attribute groupby conjunctive                      0.056667   
                         disjunctive                          0.11   
Multi attribute groupby  conjunctive                         0.088   
                         disjunctive                          0.11   
3 clusters               conjunctive                           0.0   
                         disjunctive                           0.0   
15 clusters              conjunctive                      0.054286   
                         disjunctive                      0.054286   
Average                  conjunctive                      0.105803   
                         disjunctive                      0.123946   

                                          % Explanation Not Found  \
Test                     Explanation Type                           
Full dataset             conjunctive                         50.0   
                         disjunctive                         50.0   
Numeric only             conjunctive                         50.0   
                         disjunctive                         50.0   
Categorical only         conjunctive                         50.0   
                         disjunctive                         50.0   
Single attribute groupby conjunctive                    66.666667   
                         disjunctive                    83.333333   
Multi attribute groupby  conjunctive                         90.0   
                         disjunctive                         90.0   
3 clusters               conjunctive                            0   
                         disjunctive                            0   
15 clusters              conjunctive                         40.0   
                         disjunctive                         40.0   
Average                  conjunctive                     49.52381   
                         disjunctive                    51.904762   

                                          Average Explanation Length  \
Test                     Explanation Type                              
Full dataset             conjunctive                            1.25   
                         disjunctive                             1.0   
Numeric only             conjunctive                            1.25   
                         disjunctive                         

## Spotify dataset

In [30]:
spotify = pd.read_csv(r"..\Examples\Datasets\spotify_all.csv")
spotify

,acousticness,artists,danceability,energy,explicit,id,instrumentalness,key,liveness,loudness,...,name,popularity,speechiness,tempo,valence,year,decade,popularity_score,main_artist,duration_minutes
0,0.991000,['Mamie Smith'],0.598,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,...,Keep A Song In Your Soul,12,0.0936,149.976,0.6340,1920,1920,10,Mamie Smith,2.805550
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,...,I Put A Spell On You,7,0.0534,86.889,0.9500,1920,1920,0,Screamin' Jay Hawkins,2.503333
2,0.993000,['Mamie Smith'],0.647,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,...,Golfing Papa,4,0.1740,97.600,0.6890,1920,1920,0,Mamie Smith,2.730450
3,0.000173,['Oscar Velazquez'],0.730,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,...,True House Music - Xavier Santos & Carlos Gomi...,17,0.0425,127.997,0.0422,1920,1920,10,Oscar Velazquez,7.034783
4,0.295000,['Mixe'],0.704,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,...,Xuniverxe,2,0.0768,122.076,0.2990,1920,1920,0,Mixe,2.753733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174384,0.009170,"['DJ Combo', 'Sander-7', 'Tony T']",0.792,0.866,0,46LhBf6TvYjZU2SMvGZAbn,0.000060,6,0.1780,-5.089,...,The One,0,0.0356,125.972,0.1860,2020,2020,0,DJ Combo,2.460250
174385,0.795000,['Alessia Cara'],0.429,0.211,0,7tue2Wemjd0FZzRtDrQFZd,0.000000,4,0.1960,-11.665,...,A Little More,0,0.0360,94.710,0.2280,2021,2020,0,Alessia Cara,2.412000
174386,0.806000,['Roger Fly'],0.671,0.589,0,48Qj61hOdYmUCFJbpQ29Ob,0.920000,4,0.1130,-12.393,...,Together,0,0.0282,108.058,0.7140,2020,2020,0,Roger Fly,3.635783
174387,0.920000,['Taylor Swift'],0.462,0.240,1,1gcyHQpBQ1lfXGdhZmWrHP,0.000000,0,0.1130,-12.077,...,champagne problems,69,0.0377,171.319,0.3200,2021,2020,60,Taylor Swift,4.066667


In [31]:
spotify_comparison_df = pd.DataFrame(columns=columns, index=index)

### Full dataset test

In [32]:
start_time = time.time()
conj_res = spotify.explain(explainer='many to one', labels='explicit', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, 'Full dataset', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.755469560623169


Coverage  \
Group / Cluster Explanation                                                   
0               0.0 <= speechiness <= 0.16166666666666665              0.91   
                0.0 <= speechiness <= 0.485                            0.97   
                0.0 <= speechiness <= 0.2425                           0.94   
                0.0 <= speechiness <= 0.3233333333333333 AND 0 ...     0.78   
                0.0 <= speechiness <= 0.3233333333333333               0.95   
                0.0 <= speechiness <= 0.194 AND 0 <= popularity...     0.75   
1               No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               0.0 <= speechiness <= 0.16166666666666665                      0.03  
                0.0 <= speechiness <= 0.485                                    0.06  
                0.0 <= speechiness <= 0.2425                                   0.04  
                0.0 <= speechiness <= 0.3233333333333333 AND 0 ...             0.02  
                0.0 <= speechiness <= 0.3233333333333333                       0.05  
                0.0 <= speechiness <= 0.194 AND 0 <= popularity...             0.01  
1               No explanation found                                            NaN

In [33]:
start_time = time.time()
disj_res = spotify.explain(explainer='many to one', labels='explicit', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, 'Full dataset', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.8857603073120117


Coverage  \
Group / Cluster Explanation                                                   
0               0.0 <= popularity <= 44.0 OR 0.0 <= speechiness...      1.0   
                0.0 <= speechiness <= 0.16166666666666665 OR 0 ...     0.99   
                0.0 <= speechiness <= 0.2425                           0.94   
                0.0 <= speechiness <= 0.3233333333333333               0.95   
                0.0 <= popularity <= 44.0 OR 0.0 <= speechiness...     0.99   
                0.0 <= speechiness <= 0.2425 OR 0 <= popularity...      1.0   
                0.0 <= speechiness <= 0.16166666666666665              0.91   
                0.0 <= popularity <= 44.0 OR 0.0 <= speechiness...     0.99   
                0.0 <= speechiness <= 0.485                            0.97   
1               No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               0.0 <= popularity <= 44.0 OR 0.0 <= speechiness...             0.06  
                0.0 <= speechiness <= 0.16166666666666665 OR 0 ...             0.05  
                0.0 <= speechiness <= 0.2425                                   0.04  
                0.0 <= speechiness <= 0.3233333333333333                       0.05  
                0.0 <= popularity <= 44.0 OR 0.0 <= speechiness...             0.05  
                0.0 <= speechiness <= 0.2425 OR 0 <= popularity...             0.06  
                0.0 <= speechiness <= 0.16166666666666665                      0.03  
                0.0 <= popularity <= 44.0 OR 0.0 <= speechiness...             0.05  
                0.0 <= speechiness <= 0.485                                    0.06  
1               No explanation found                                            NaN

### Numeric only and categorical only tests

In [34]:
numeric_only = spotify[['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']]
categorical_only = spotify[['name', 'main_artist', 'year']]
labels = spotify['explicit']

In [35]:
start_time = time.time()
conj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, 'Numeric only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.5948865413665771


Coverage  \
Group / Cluster Explanation                                          
0               0.0 <= speechiness <= 0.485                   0.97   
                0.0 <= speechiness <= 0.2425                  0.94   
                0.0 <= speechiness <= 0.3233333333333333      0.95   
                0.0 <= speechiness <= 0.16166666666666665     0.91   
1               No explanation found                           NaN   

                                                          Separation Error  
Group / Cluster Explanation                                                 
0               0.0 <= speechiness <= 0.485                           0.06  
                0.0 <= speechiness <= 0.2425                          0.04  
                0.0 <= speechiness <= 0.3233333333333333              0.05  
                0.0 <= speechiness <= 0.16166666666666665             0.03  
1               No explanation found                                   NaN

In [36]:
start_time = time.time()
disj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, 'Numeric only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.7212879657745361


Coverage  \
Group / Cluster Explanation                                                   
0               0.0 <= speechiness <= 0.3233333333333333 OR 0.0...      1.0   
                0.0 <= speechiness <= 0.16166666666666665              0.91   
                0.0 <= instrumentalness <= 0.4995 OR 0.0 <= spe...      1.0   
                0.0 <= speechiness <= 0.485                            0.97   
                0.0 <= speechiness <= 0.3233333333333333 OR 0.0...      1.0   
                0.0 <= instrumentalness <= 0.24975 OR 0.0 <= sp...      1.0   
                0.0 <= speechiness <= 0.2425                           0.94   
                0.0 <= speechiness <= 0.3233333333333333               0.95   
                0.0 <= instrumentalness <= 0.333 OR 0.0 <= spee...      1.0   
1               No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               0.0 <= speechiness <= 0.3233333333333333 OR 0.0...             0.07  
                0.0 <= speechiness <= 0.16166666666666665                      0.03  
                0.0 <= instrumentalness <= 0.4995 OR 0.0 <= spe...             0.07  
                0.0 <= speechiness <= 0.485                                    0.06  
                0.0 <= speechiness <= 0.3233333333333333 OR 0.0...             0.07  
                0.0 <= instrumentalness <= 0.24975 OR 0.0 <= sp...             0.07  
                0.0 <= speechiness <= 0.2425                                   0.04  
                0.0 <= speechiness <= 0.3233333333333333                       0.05  
                0.0 <= instrumentalness <= 0.333 OR 0.0 <= spee...             0.07  
1               No explanation found                                            NaN

In [37]:
start_time = time.time()
conj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, 'Categorical only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.32207226753234863


,,Coverage,Separation Error
Group / Cluster,Explanation,,
0,name == infrequent_sklearn,1.0,0.07
1,No explanation found,NaN,NaN


In [38]:
start_time = time.time()
disj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, 'Categorical only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.28539109230041504


,,Coverage,Separation Error
Group / Cluster,Explanation,,
0,name == infrequent_sklearn,1.0,0.07
1,No explanation found,NaN,NaN


### Groupby tests

In [39]:
single_attribute_groupby = spotify.groupby('mode').mean()

In [40]:
start_time = time.time()
conj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, 'Single attribute groupby', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.9400739669799805


Coverage  \
Group / Cluster Explanation                                                   
1               0.07324999943375587 <= acousticness <= 0.996           0.78   
                0.0 <= speechiness <= 0.485                            0.95   
                0.0 <= speechiness <= 0.3233333333333333 AND 0....     0.73   
0               No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
1               0.07324999943375587 <= acousticness <= 0.996                   0.28  
                0.0 <= speechiness <= 0.485                                     0.3  
                0.0 <= speechiness <= 0.3233333333333333 AND 0....             0.27  
0               No explanation found                                            NaN

In [41]:
start_time = time.time()
disj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, 'Single attribute groupby', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 1.0879826545715332


Coverage  \
Group / Cluster Explanation                                                   
1               0.07324999943375587 <= acousticness <= 0.996           0.78   
                0.07324999943375587 <= acousticness <= 0.996 OR...      1.0   
                0.0 <= speechiness <= 0.485                            0.95   
0               No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
1               0.07324999943375587 <= acousticness <= 0.996                   0.28  
                0.07324999943375587 <= acousticness <= 0.996 OR...              0.3  
                0.0 <= speechiness <= 0.485                                     0.3  
0               No explanation found                                            NaN

### Clustering test

#### 3 clusters

In [42]:
# The spotify dataset is too big to cluster all columns with get_dummies (I tried, got a MemoryError, saying it is unable to allocate 22.3 GiB)
cols = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']

In [43]:
cluster_labels = KMeans(n_clusters=3).fit_predict(spotify[cols])

In [44]:
start_time = time.time()
conj_res = spotify.explain(explainer='many to one', labels=cluster_labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, '3 clusters', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 1.1946170330047607


Coverage  \
Group / Cluster Explanation                                                  
0               0.0 <= tempo <= 106.38400268554688                    0.99   
                0.0 <= tempo <= 109.723                                1.0   
1               106.68099975585938 <= tempo <= 219.446                 1.0   
                109.72300000000001 <= tempo <= 146.29733333333334     0.92   
                106.78049850463867 <= tempo <= 219.446                 1.0   
                109.723 <= tempo <= 164.5845                          0.93   
2               147.2570037841797 <= tempo <= 219.446                  1.0   

                                                                  Separation Error  
Group / Cluster Explanation                                                         
0               0.0 <= tempo <= 106.38400268554688                             0.0  
                0.0 <= tempo <= 109.723                                       0.07  
1               106.68099975585938 <= tempo <= 219.446                        0.25  
                109.72300000000001 <= tempo <= 146.29733333333334              0.0  
                106.78049850463867 <= tempo <= 219.446                        0.25  
                109.723 <= tempo <= 164.5845                                  0.13  
2               147.2570037841797 <= tempo <= 219.446                          0.0

In [45]:
start_time = time.time()
disj_res = spotify.explain(explainer='many to one', labels=cluster_labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, '3 clusters', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 1.3401148319244385


Coverage  \
Group / Cluster Explanation                                                  
0               0.0 <= tempo <= 109.723                                1.0   
                0.0 <= tempo <= 106.38400268554688                    0.99   
1               106.78049850463867 <= tempo <= 219.446                 1.0   
                109.72300000000001 <= tempo <= 146.29733333333334     0.92   
                109.723 <= tempo <= 164.5845                          0.93   
                106.68099975585938 <= tempo <= 219.446                 1.0   
2               147.2570037841797 <= tempo <= 219.446                  1.0   

                                                                  Separation Error  
Group / Cluster Explanation                                                         
0               0.0 <= tempo <= 109.723                                       0.07  
                0.0 <= tempo <= 106.38400268554688                             0.0  
1               106.78049850463867 <= tempo <= 219.446                        0.25  
                109.72300000000001 <= tempo <= 146.29733333333334              0.0  
                109.723 <= tempo <= 164.5845                                  0.13  
                106.68099975585938 <= tempo <= 219.446                        0.25  
2               147.2570037841797 <= tempo <= 219.446                          0.0

#### 15 clusters

In [46]:
cluster_labels = KMeans(n_clusters=15).fit_predict(spotify[cols])

In [47]:
start_time = time.time()
conj_res = spotify.explain(explainer='many to one', labels=cluster_labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, '15 clusters', 'conjunctive', conj_res, conj_time)
conj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Conjunctive explanation runtime: 4.911241292953491


Coverage  \
Group / Cluster Explanation                                                   
0               112.581 <= tempo <= 125.547 AND -20.3665 <= lou...     0.96   
                112.581 <= tempo <= 125.547                            0.97   
3               87.1984977722168 <= tempo <= 100.50500106811523...     0.95   
                86.93149948120117 <= tempo <= 100.50500106811523        1.0   
                87.762 <= tempo <= 99.814                              0.93   
                86.93149948120117 <= tempo <= 100.5050010681152...     0.93   
                86.93149948120117 <= tempo <= 100.5050010681152...      1.0   
                87.1984977722168 <= tempo <= 100.50500106811523...     0.94   
                87.762 <= tempo <= 99.814 AND -15.7255001068115...     0.88   
                86.93149948120117 <= tempo <= 100.5050010681152...     0.97   
                87.1984977722168 <= tempo <= 100.50500106811523...     0.92   
                87.1984977722168 <= tempo <= 100.50500106811523...     0.98   
                87.762 <= tempo <= 99.814 AND -15.0675001144409...     0.86   
                86.93149948120117 <= tempo <= 100.5050010681152...     0.95   
                87.1984977722168 <= tempo <= 100.50500106811523...     0.96   
4               120.034 <= tempo <= 137.06 AND -10.368 <= loudn...     0.76   
                120.034 <= tempo <= 137.06 AND -13.529999999999...     0.98   
5               95.03999710083008 <= tempo <= 113.8445014953613...     0.74   
                95.03999710083008 <= tempo <= 113.8445014953613...     0.88   
                95.03999710083008 <= tempo <= 113.8445014953613...     0.94   
6               100.50500106811523 <= tempo <= 112.621997833251...     0.96   
                100.50500106811523 <= tempo <= 112.621997833251...     0.99   
                100.50500106811523 <= tempo <= 112.621997833251...      0.9   
                100.50500106811523 <= tempo <= 112.621997833251...     0.94   
8               133.0308 <= tempo <= 154.0424                           1.0   
12              75.0459976196289 <= tempo <= 86.85100173950195 ...     0.82   
                69.99600000000001 <= tempo <= 87.50566666666667...     0.82   
                69.99600000000001 <= tempo <= 87.50566666666667...      1.0   
                75.0459976196289 <= tempo <= 86.85100173950195 ...     0.71   
                75.0459976196289 <= tempo <= 86.93149948120117 ...     0.71   
                75.0459976196289 <= tempo <= 86.93149948120117 ...     0.82   
                69.99600000000001 <= tempo <= 87.50566666666667...     0.99   
                69.99600000000001 <= tempo <= 87.50566666666667...     0.94   
                69.99600000000001 <= tempo <= 87.50566666666667...     0.95   
                75.0459976196289 <= tempo <= 86.85100173950195 ...     0.83   
13              160.44000244140625 <= tempo <= 175.054                 0.98   
                157.54433333333333 <= tempo <= 175.054                  1.0   
14              75.0459976196289 <= tempo <= 91.51750183105469 ...     0.82   
                69.99600000000001 <= tempo <= 94.313 AND -40.87...     0.97   
                69.99600000000001 <= tempo <= 94.313 AND -40.87...      1.0   
                75.0459976196289 <= tempo <= 91.51750183105469 ...     0.88   
                69.99600000000001 <= tempo <= 94.313 AND -40.87...     0.91   
2               No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               112.581 <= tempo <= 125.547 AND -20.3665 <= lou...              0.2  
                112.581 <= tempo <= 125.547                                    0.21  
3               87.1984977722168 <= tempo <= 100.50500106811523...             0.03  
                86.93149948120117 <= tempo <= 100.50500106811523               0.

In [48]:
start_time = time.time()
disj_res = spotify.explain(explainer='many to one', labels=cluster_labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, '15 clusters', 'disjunctive', disj_res, disj_time)
disj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Disjunctive explanation runtime: 4.6106226444244385


Coverage  \
Group / Cluster Explanation                                                 
0               112.581 <= tempo <= 125.547                          0.97   
3               87.762 <= tempo <= 99.814                            0.93   
                86.93149948120117 <= tempo <= 100.50500106811523      1.0   
8               133.0308 <= tempo <= 154.0424                         1.0   
13              160.44000244140625 <= tempo <= 175.054               0.98   
                157.54433333333333 <= tempo <= 175.054                1.0   
2               No explanation found                                  NaN   
4               No explanation found                                  NaN   
5               No explanation found                                  NaN   
6               No explanation found                                  NaN   
12              No explanation found                                  NaN   
14              No explanation found                                  NaN   

                                                                 Separation Error  
Group / Cluster Explanation                                                        
0               112.581 <= tempo <= 125.547                                  0.21  
3               87.762 <= tempo <= 99.814                                    0.19  
                86.93149948120117 <= tempo <= 100.50500106811523             0.21  
8               133.0308 <= tempo <= 154.0424                                 0.3  
13              160.44000244140625 <= tempo <= 175.054                        0.0  
                157.54433333333333 <= tempo <= 175.054                       0.17  
2               No explanation found                                          NaN  
4               No explanation found                                          NaN  
5               No explanation found                                          NaN  
6               No explanation found                                          NaN  
12              No explanation found                                          NaN  
14              No explanation found                                          NaN

### Summary of results

In [49]:
conj_averages = spotify_comparison_df.loc[:, 'conjunctive', :].mean()
disj_averages = spotify_comparison_df.loc[:, 'disjunctive', :].mean()
spotify_comparison_df.loc[('Average', 'conjunctive'), :] = conj_averages
spotify_comparison_df.loc[('Average', 'disjunctive'), :] = disj_averages

In [50]:
spotify_comparison_df

Average Coverage  \
Test                     Explanation Type                    
Full dataset             conjunctive              0.883333   
                         disjunctive              0.971111   
Numeric only             conjunctive                0.9425   
                         disjunctive              0.974444   
Categorical only         conjunctive                   1.0   
                         disjunctive                   1.0   
Single attribute groupby conjunctive                  0.82   
                         disjunctive                  0.91   
3 clusters               conjunctive              0.977143   
                         disjunctive              0.977143   
15 clusters              conjunctive              0.915238   
                         disjunctive                  0.98   
Average                  conjunctive              0.923036   
                         disjunctive              0.968783   

                                          Average Separation Error  \
Test                     Explanation Type                            
Full dataset             conjunctive                         0.035   
                         disjunctive                          0.05   
Numeric only             conjunctive                         0.045   
                         disjunctive                      0.058889   
Categorical only         conjunctive                          0.07   
                         disjunctive                          0.07   
Single attribute groupby conjunctive                      0.283333   
                         disjunctive                      0.293333   
3 clusters               conjunctive                           0.1   
                         disjunctive                           0.1   
15 clusters              conjunctive                      0.095476   
                         disjunctive                          0.18   
Average                  conjunctive                      0.104802   
                         disjunctive                       0.12537   

                                          % Explanation Not Found  \
Test                     Explanation Type                           
Full dataset             conjunctive                         50.0   
                         disjunctive                         50.0   
Numeric only             conjunctive                         50.0   
                         disjunctive                         50.0   
Categorical only         conjunctive                         50.0   
                         disjunctive                         50.0   
Single attribute groupby conjunctive                         50.0   
                         disjunctive                         50.0   
3 clusters               conjunctive                            0   
                         disjunctive                            0   
15 clusters              conjunctive                         10.0   
                         disjunctive                         60.0   
Average                  conjunctive                         35.0   
                         disjunctive                    43.333333   

                                          Average Explanation Length  \
Test                     Explanation Type                              
Full dataset             conjunctive                        1.333333   
                         disjunctive                        1.555556   
Numeric only             conjunctive                             1.0   
                         disjunctive                        1.555556   
Categorical only         conjunctive                             1.0   
                         disjunctive                             1.0   
Single attribute groupby conjunctive                        1.333333   
                         disjunctive                        1.333333   
3 clusters               conjunctive                             1.0   
                         disjun

## House Prices dataset

In [51]:
houses = pd.read_csv(r"..\Examples\Datasets\houses.csv")
houses = houses[['MSSubClass', 'LotArea', 'Street', 'LotConfig', 'LandSlope', 'Neighborhood', 'BldgType', 'HouseStyle',
'OverallQual', 'OverallCond', 'YearBuilt', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'ExterQual', 'Foundation', 'BsmtQual',
'CentralAir', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'GarageCars', 'PoolArea', 'YrSold', 'SalePrice']]

In [52]:
houses_comparison_df = pd.DataFrame(columns=columns, index=index)

### Full dataset test

In [53]:
start_time = time.time()
conj_res = houses.explain(explainer='many to one', labels='HouseStyle', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(houses_comparison_df, 'Full dataset', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 2.317922353744507


Coverage  \
Group / Cluster Explanation                                                   
2Story          671.0 <= 2ndFlrSF <= 2065 AND 55.0 <= MSSubClas...     0.85   
                545.5 <= 2ndFlrSF <= 2065 AND 55.0 <= MSSubClas...     0.96   
                55.0 <= MSSubClass <= 72.5                              0.8   
                200.0 <= 2ndFlrSF <= 2065 AND 60 <= MSSubClass ...     0.99   
                671.0 <= 2ndFlrSF <= 2065                              0.85   
                600 <= 2ndFlrSF <= 2065                                0.91   
                545.5 <= 2ndFlrSF <= 2065 AND 60 <= MSSubClass ...     0.96   
                200.0 <= 2ndFlrSF <= 2065                               1.0   
                671.0 <= 2ndFlrSF <= 2065 AND 60 <= MSSubClass ...     0.85   
                545.5 <= 2ndFlrSF <= 2065                              0.97   
                200.0 <= 2ndFlrSF <= 2065 AND 55.0 <= MSSubClas...     0.99   
1Story          2ndFlrSF == 0                                           1.0   
                0.0 <= 2ndFlrSF <= 344.1666666666667 AND 20 <= ...     0.83   
                20 <= MSSubClass <= 25.0                               0.74   
                0 <= 2ndFlrSF <= 728 AND 20 <= MSSubClass <= 30        0.83   
                2ndFlrSF == 0 AND 20 <= MSSubClass <= 42.5             0.83   
                0.0 <= 2ndFlrSF <= 688.3333333333334 AND 20 <= ...     0.83   
                0 <= 2ndFlrSF <= 55.0 AND 20 <= MSSubClass <= 30       0.83   
                0 <= 2ndFlrSF <= 456 AND 20 <= MSSubClass <= 42.5      0.83   
                0 <= 2ndFlrSF <= 600 AND 20 <= MSSubClass <= 30        0.83   
                0.0 <= 2ndFlrSF <= 516.25 AND 20 <= MSSubClass ...     0.83   
                20 <= MSSubClass <= 42.5                               0.83   
                0.0 <= 2ndFlrSF <= 413.0 AND 20 <= MSSubClass <...     0.83   
                0 <= 2ndFlrSF <= 545.5 AND 20 <= MSSubClass <= ...     0.83   
                0 <= 2ndFlrSF <= 55.0                                   1.0   
                MSSubClass == 20                                       0.74   
                0.0 <= 2ndFlrSF <= 344.1666666666667 AND 20 <= ...     0.83   
                2ndFlrSF == 0 AND 20 <= MSSubClass <= 30               0.83   
                0.0 <= 2ndFlrSF <= 688.3333333333334 AND 20 <= ...     0.83   
                0 <= 2ndFlrSF <= 728 AND 20 <= MSSubClass <= 42.5      0.83   
                0 <= 2ndFlrSF <= 456 AND 20 <= MSSubClass <= 30        0.83   
                0 <= 2ndFlrSF <= 55.0 AND 20 <= MSSubClass <= 42.5     0.83   
                2ndFlrSF == 0                                           1.0   
                20 <= MSSubClass <= 30                                 0.83   
                0 <= 2ndFlrSF <= 600 AND 20 <= MSSubClass <= 42.5      0.83   
                0.0 <= 2ndFlrSF <= 516.25 AND 20 <= MSSubClass ...     0.83   
                0 <= 2ndFlrSF <= 545.5 AND 20 <= MSSubClass <= 30      0.83   
                0.0 <= 2ndFlrSF <= 413.0 AND 20 <= MSSubClass <...     0.83   
                MSSubClass == 20                                       0.74   
1.5Fin          MSSubClass == 50                                       0.92   
1.5Unf          42.5 <= MSSubClass <= 47.5                             0.86   
                MSSubClass == 45                                       0.86   
SLvl            MSSubClass == 80                                       0.89   
2.5Unf          334.0 <= GrLivArea <= 2988.0 AND MSSubClass == 75      0.82   
2.5Fin          No explanation found                                    NaN   
SFoyer          No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
2Story          671.0 <= 2ndFlrSF <= 2065 AND 55.0 <= MSSubClas...             0.07  
                

In [54]:
start_time = time.time()
disj_res = houses.explain(explainer='many to one', labels='HouseStyle', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(houses_comparison_df, 'Full dataset', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 1.0194993019104004


Coverage  \
Group / Cluster Explanation                                                   
2Story          200.0 <= 2ndFlrSF <= 2065                               1.0   
                545.5 <= 2ndFlrSF <= 2065 OR 55.0 <= MSSubClass...     0.98   
                545.5 <= 2ndFlrSF <= 2065                              0.97   
                600 <= 2ndFlrSF <= 2065                                0.91   
                55.0 <= MSSubClass <= 72.5                              0.8   
                671.0 <= 2ndFlrSF <= 2065                              0.85   
                671.0 <= 2ndFlrSF <= 2065 OR 55.0 <= MSSubClass...     0.91   
                55.0 <= MSSubClass <= 72.5 OR 600 <= 2ndFlrSF <...     0.95   
1Story          20 <= MSSubClass <= 20                                 0.74   
                20 <= MSSubClass <= 42.5                               0.83   
                0 <= 2ndFlrSF <= 0                                      1.0   
                0 <= 2ndFlrSF <= 55.0                                   1.0   
                20 <= MSSubClass <= 30                                 0.83   
                20 <= MSSubClass <= 25.0                               0.74   
1.5Fin          42.5 <= MSSubClass <= 55.0                             0.92   
1.5Unf          42.5 <= MSSubClass <= 47.5                             0.86   
2.5Fin          No explanation found                                    NaN   
2.5Unf          No explanation found                                    NaN   
SFoyer          No explanation found                                    NaN   
SLvl            No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
2Story          200.0 <= 2ndFlrSF <= 2065                                      0.29  
                545.5 <= 2ndFlrSF <= 2065 OR 55.0 <= MSSubClass...              0.2  
                545.5 <= 2ndFlrSF <= 2065                                       0.2  
                600 <= 2ndFlrSF <= 2065                                        0.17  
                55.0 <= MSSubClass <= 72.5                                     0.01  
                671.0 <= 2ndFlrSF <= 2065                                      0.14  
                671.0 <= 2ndFlrSF <= 2065 OR 55.0 <= MSSubClass...             0.13  
                55.0 <= MSSubClass <= 72.5 OR 600 <= 2ndFlrSF <...             0.17  
1Story          20 <= MSSubClass <= 20                                          0.0  
                20 <= MSSubClass <= 42.5                                       0.01  
                0 <= 2ndFlrSF <= 0                                             0.13  
                0 <= 2ndFlrSF <= 55.0                                          0.13  
                20 <= MSSubClass <= 30                                         0.01  
                20 <= MSSubClass <= 25.0                                        0.0  
1.5Fin          42.5 <= MSSubClass <= 55.0                                      0.1  
1.5Unf          42.5 <= MSSubClass <= 47.5                                      0.0  
2.5Fin          No explanation found                                            NaN  
2.5Unf          No explanation found                                            NaN  
SFoyer          No explanation found                                            NaN  
SLvl            No explanation found                                            NaN

### Numeric only and categorical only tests

In [55]:
numeric_only = houses[['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'GarageCars', 'PoolArea', 'SalePrice']]
categorical_only = houses[['MSSubClass', 'Street', 'LotConfig', 'LandSlope', 'Neighborhood', 'BldgType', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'ExterQual', 'Foundation', 'BsmtQual', 'CentralAir', 'YrSold']]
labels = houses['HouseStyle']

In [56]:
start_time = time.time()
conj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(houses_comparison_df, 'Numeric only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 1.8937678337097168


Coverage  \
Group / Cluster Explanation                                                   
2Story          545.5 <= 2ndFlrSF <= 2065 AND 334.0 <= GrLivAre...     0.95   
                671.0 <= 2ndFlrSF <= 2065                              0.85   
                671.0 <= 2ndFlrSF <= 2065 AND 1964.0 <= YearBui...      0.7   
                671.0 <= 2ndFlrSF <= 2065 AND 334.0 <= GrLivAre...     0.83   
                545.5 <= 2ndFlrSF <= 2065 AND 1968.5 <= YearBui...     0.75   
                200.0 <= 2ndFlrSF <= 2065 AND 1964.0 <= YearBui...     0.81   
                600 <= 2ndFlrSF <= 2065                                0.91   
                545.5 <= 2ndFlrSF <= 2065 AND 1941.0 <= YearBui...     0.82   
                200.0 <= 2ndFlrSF <= 2065                               1.0   
                545.5 <= 2ndFlrSF <= 2065                              0.97   
                545.5 <= 2ndFlrSF <= 2065 AND 1964.0 <= YearBui...     0.79   
                688.3333333333334 <= 2ndFlrSF <= 1376.666666666...     0.78   
1Story          2ndFlrSF == 0                                           1.0   
                2ndFlrSF == 0 AND 2776.0 <= LotArea <= 215245           1.0   
                0 <= 2ndFlrSF <= 55.0 AND 2776.0 <= LotArea <= ...      1.0   
                0 <= 2ndFlrSF <= 55.0                                   1.0   
                2ndFlrSF == 0 AND 2776.0 <= LotArea <= 215245 A...     0.91   
                2ndFlrSF == 0                                           1.0   
1.5Fin          No explanation found                                    NaN   
1.5Unf          No explanation found                                    NaN   
2.5Fin          No explanation found                                    NaN   
2.5Unf          No explanation found                                    NaN   
SFoyer          No explanation found                                    NaN   
SLvl            No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
2Story          545.5 <= 2ndFlrSF <= 2065 AND 334.0 <= GrLivAre...             0.19  
                671.0 <= 2ndFlrSF <= 2065                                      0.14  
                671.0 <= 2ndFlrSF <= 2065 AND 1964.0 <= YearBui...             0.04  
                671.0 <= 2ndFlrSF <= 2065 AND 334.0 <= GrLivAre...             0.13  
                545.5 <= 2ndFlrSF <= 2065 AND 1968.5 <= YearBui...             0.06  
                200.0 <= 2ndFlrSF <= 2065 AND 1964.0 <= YearBui...             0.08  
                600 <= 2ndFlrSF <= 2065                                        0.17  
                545.5 <= 2ndFlrSF <= 2065 AND 1941.0 <= YearBui...             0.12  
                200.0 <= 2ndFlrSF <= 2065                                      0.29  
                545.5 <= 2ndFlrSF <= 2065                                       0.2  
                545.5 <= 2ndFlrSF <= 2065 AND 1964.0 <= YearBui...             0.07  
                688.3333333333334 <= 2ndFlrSF <= 1376.666666666...             0.12  
1Story          2ndFlrSF == 0                                                  0.13  
                2ndFlrSF == 0 AND 2776.0 <= LotArea <= 215245                  0.12  
                0 <= 2ndFlrSF <= 55.0 AND 2776.0 <= LotArea <= ...             0.12  
                0 <= 2ndFlrSF <= 55.0                                          0.13  
                2ndFlrSF == 0 AND 2776.0 <= LotArea <= 215245 A...             0.11  
                2ndFlrSF == 0                                                  0.13  
1.5Fin          No explanation found                                            NaN  
1.5Unf          No explanation found                                            NaN  
2.5Fin          No explanation found                                            NaN  
2.5Unf          No explanation fou

In [57]:
start_time = time.time()
disj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(houses_comparison_df, 'Numeric only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.799609899520874


Coverage  \
Group / Cluster Explanation                                                   
2Story          200.0 <= 2ndFlrSF <= 2065                               1.0   
                688.3333333333334 <= 2ndFlrSF <= 1376.666666666...     0.78   
                600 <= 2ndFlrSF <= 2065                                0.91   
                671.0 <= 2ndFlrSF <= 2065                              0.85   
                545.5 <= 2ndFlrSF <= 2065                              0.97   
1Story          0 <= 2ndFlrSF <= 55.0                                   1.0   
                0 <= 2ndFlrSF <= 0                                      1.0   
1.5Fin          No explanation found                                    NaN   
1.5Unf          No explanation found                                    NaN   
2.5Fin          No explanation found                                    NaN   
2.5Unf          No explanation found                                    NaN   
SFoyer          No explanation found                                    NaN   
SLvl            No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
2Story          200.0 <= 2ndFlrSF <= 2065                                      0.29  
                688.3333333333334 <= 2ndFlrSF <= 1376.666666666...             0.12  
                600 <= 2ndFlrSF <= 2065                                        0.17  
                671.0 <= 2ndFlrSF <= 2065                                      0.14  
                545.5 <= 2ndFlrSF <= 2065                                       0.2  
1Story          0 <= 2ndFlrSF <= 55.0                                          0.13  
                0 <= 2ndFlrSF <= 0                                             0.13  
1.5Fin          No explanation found                                            NaN  
1.5Unf          No explanation found                                            NaN  
2.5Fin          No explanation found                                            NaN  
2.5Unf          No explanation found                                            NaN  
SFoyer          No explanation found                                            NaN  
SLvl            No explanation found                                            NaN

In [58]:
start_time = time.time()
conj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(houses_comparison_df, 'Categorical only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.45153117179870605


Coverage Separation Error
Group / Cluster Explanation                                         
2Story          55.0 <= MSSubClass <= 72.5      0.8             0.01
                50 <= MSSubClass <= 70         0.81             0.28
1Story          20 <= MSSubClass <= 30         0.83             0.01
                20 <= MSSubClass <= 42.5       0.83             0.01
                20 <= MSSubClass <= 25.0       0.74              0.0
                MSSubClass == 20               0.74              0.0
                MSSubClass == 20               0.74              0.0
1.5Fin          MSSubClass == 50               0.92             0.02
1.5Unf          42.5 <= MSSubClass <= 47.5     0.86              0.0
                MSSubClass == 45               0.86              0.0
SLvl            MSSubClass == 80               0.89              0.0
2.5Fin          No explanation found            NaN              NaN
2.5Unf          No explanation found            NaN              NaN
SFoyer          No explanation found            NaN              NaN

In [59]:
start_time = time.time()
disj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(houses_comparison_df, 'Categorical only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.48761868476867676


Coverage Separation Error
Group / Cluster Explanation                                         
2Story          50 <= MSSubClass <= 70         0.81             0.28
                55.0 <= MSSubClass <= 72.5      0.8             0.01
1Story          20 <= MSSubClass <= 30         0.83             0.01
                20 <= MSSubClass <= 42.5       0.83             0.01
                20 <= MSSubClass <= 20         0.74              0.0
                20 <= MSSubClass <= 25.0       0.74              0.0
1.5Fin          42.5 <= MSSubClass <= 55.0     0.92              0.1
1.5Unf          42.5 <= MSSubClass <= 47.5     0.86              0.0
2.5Fin          No explanation found            NaN              NaN
2.5Unf          No explanation found            NaN              NaN
SFoyer          No explanation found            NaN              NaN
SLvl            No explanation found            NaN              NaN

### Groupby tests

In [60]:
single_attribute_groupby = houses.groupby('ExterQual').mean()
multi_attribute_groupby = houses.groupby(['OverallQual', 'ExterQual']).mean()

In [61]:
start_time = time.time()
conj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(houses_comparison_df, 'Single attribute groupby', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.8108689785003662


Coverage  \
Group / Cluster Explanation                                                   
Gd              1984.5 <= YearBuilt <= 2010                            0.86   
                1982.4 <= YearBuilt <= 2010.0                          0.87   
                1980.5 <= YearBuilt <= 2010 AND 7.0 <= OverallQ...     0.76   
                1980.5 <= YearBuilt <= 2010 AND 6.5 <= OverallQ...     0.76   
                6.4 <= OverallQual <= 8.2                              0.76   
                1987.0 <= YearBuilt <= 2010.0 AND 5.5 <= Overal...     0.82   
                1980.5 <= YearBuilt <= 2010 AND 171625.0 <= Sal...     0.79   
                1980.5 <= YearBuilt <= 2010 AND 6.4 <= OverallQ...     0.72   
                1980.5 <= YearBuilt <= 2010 AND 6 <= OverallQua...     0.88   
                1982.4 <= YearBuilt <= 2010.0 AND 6.4 <= Overal...     0.72   
                1987.0 <= YearBuilt <= 2010.0 AND 6 <= OverallQ...     0.82   
                1975.5 <= YearBuilt <= 2010.0 AND 6.5 <= Overal...     0.77   
                1982.4 <= YearBuilt <= 2010.0 AND 6.5 <= Overal...     0.76   
                1980.5 <= YearBuilt <= 2010 AND 7.0 <= OverallQ...     0.74   
                1975.5 <= YearBuilt <= 2010.0 AND 7.0 <= Overal...     0.73   
                1987.0 <= YearBuilt <= 2010.0 AND 6 <= OverallQ...     0.85   
                1982.4 <= YearBuilt <= 2010.0 AND 7.0 <= Overal...     0.72   
                1987.0 <= YearBuilt <= 2010.0 AND 5.5 <= Overal...     0.85   
                1980.5 <= YearBuilt <= 2010 AND 6.5 <= OverallQ...     0.74   
                1964.0 <= YearBuilt <= 2010.0 AND 7.0 <= Overal...     0.74   
                1980.5 <= YearBuilt <= 2010 AND 5.5 <= OverallQ...     0.88   
                1982.4 <= YearBuilt <= 2010.0 AND 7.0 <= Overal...     0.76   
                1980.5 <= YearBuilt <= 2010 AND 7.0 <= OverallQ...     0.72   
                1980.5 <= YearBuilt <= 2010 AND 6 <= OverallQua...     0.85   
                1980.5 <= YearBuilt <= 2010 AND 5.5 <= OverallQ...     0.85   
                1964.0 <= YearBuilt <= 2010.0 AND 6.4 <= Overal...     0.74   
                1980.5 <= YearBuilt <= 2010                            0.88   
                1975.5 <= YearBuilt <= 2010.0 AND 7.0 <= Overal...     0.77   
                7.0 <= OverallQual <= 8.5                              0.76   
                1975.5 <= YearBuilt <= 2010.0 AND 6.4 <= Overal...     0.73   
TA              4.0 <= OverallQual <= 7.0 AND 34900.0 <= SalePr...     0.92   
                34900.0 <= SalePrice <= 214925.0                       0.94   
                4.0 <= OverallQual <= 7.0 AND 1872 <= YearBuilt...     0.84   
                1872 <= YearBuilt <= 1980.5                            0.86   
                4.0 <= OverallQual <= 7.0 AND 1872 <= YearBuilt...      0.8   
                1 <= OverallQual <= 6                                  0.89   
                1872 <= YearBuilt <= 1980.5 AND 34900.0 <= Sale...     0.85   
                1872 <= YearBuilt <= 1980.5 AND 34900.0 <= Sale...     0.82   
                1.5 <= OverallQual <= 6.5                              0.89   
                4.0 <= OverallQual <= 7.0                              0.97   
                1 <= OverallQual <= 6.5                                0.89   
                4.0 <= OverallQual <= 7.0 AND 34900.0 <= SalePr...     0.97   
Ex              No explanation found                                    NaN   
Fa              No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
Gd              1984.5 <= YearBuilt <= 2010                                    0.27  
                1982.4 <= YearBuilt <= 2010.0                                  0.28  
                1980.5 <= YearBuilt <= 2010 AND 7.0 <= OverallQ...             0.19  
  

In [62]:
start_time = time.time()
disj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(houses_comparison_df, 'Single attribute groupby', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.6081326007843018


Coverage  \
Group / Cluster Explanation                                                   
Gd              7.0 <= OverallQual <= 8.5                              0.76   
                6.4 <= OverallQual <= 8.2                              0.76   
                1984.5 <= YearBuilt <= 2010                            0.86   
                1982.4 <= YearBuilt <= 2010.0                          0.87   
                1980.5 <= YearBuilt <= 2010                            0.88   
TA              1872 <= YearBuilt <= 1980.5 OR 34900.0 <= SaleP...     0.98   
                1872 <= YearBuilt <= 1980.5                            0.86   
                1 <= OverallQual <= 6 OR 34900.0 <= SalePrice <...     0.97   
                1872 <= YearBuilt <= 1973 OR 4.0 <= OverallQual...      1.0   
                1.5 <= OverallQual <= 6.5                              0.89   
                1 <= OverallQual <= 6.5                                0.89   
                1.5 <= OverallQual <= 6.5 OR 34900.0 <= SalePri...     0.97   
                1 <= OverallQual <= 6                                  0.89   
                34900.0 <= SalePrice <= 214925.0                       0.94   
                1.5 <= OverallQual <= 6.5 OR 1872 <= YearBuilt ...     0.96   
                4.0 <= OverallQual <= 7.0 OR 1872 <= YearBuilt ...      1.0   
                1 <= OverallQual <= 6.5 OR 34900.0 <= SalePrice...     0.97   
                1 <= OverallQual <= 6.5 OR 1872 <= YearBuilt <=...     0.96   
                1872 <= YearBuilt <= 1973 OR 34900.0 <= SalePri...     0.97   
                1 <= OverallQual <= 6 OR 1872 <= YearBuilt <= 1...     0.96   
                4.0 <= OverallQual <= 7.0                              0.97   
Ex              No explanation found                                    NaN   
Fa              No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
Gd              7.0 <= OverallQual <= 8.5                                      0.24  
                6.4 <= OverallQual <= 8.2                                      0.24  
                1984.5 <= YearBuilt <= 2010                                    0.27  
                1982.4 <= YearBuilt <= 2010.0                                  0.28  
                1980.5 <= YearBuilt <= 2010                                    0.29  
TA              1872 <= YearBuilt <= 1980.5 OR 34900.0 <= SaleP...             0.24  
                1872 <= YearBuilt <= 1980.5                                    0.09  
                1 <= OverallQual <= 6 OR 34900.0 <= SalePrice <...             0.23  
                1872 <= YearBuilt <= 1973 OR 4.0 <= OverallQual...             0.27  
                1.5 <= OverallQual <= 6.5                                      0.12  
                1 <= OverallQual <= 6.5                                        0.12  
                1.5 <= OverallQual <= 6.5 OR 34900.0 <= SalePri...             0.23  
                1 <= OverallQual <= 6                                          0.12  
                34900.0 <= SalePrice <= 214925.0                               0.23  
                1.5 <= OverallQual <= 6.5 OR 1872 <= YearBuilt ...             0.14  
                4.0 <= OverallQual <= 7.0 OR 1872 <= YearBuilt ...             0.27  
                1 <= OverallQual <= 6.5 OR 34900.0 <= SalePrice...             0.23  
                1 <= OverallQual <= 6.5 OR 1872 <= YearBuilt <=...             0.14  
                1872 <= YearBuilt <= 1973 OR 34900.0 <= SalePri...             0.23  
                1 <= OverallQual <= 6 OR 1872 <= YearBuilt <= 1...             0.14  
                4.0 <= OverallQual <= 7.0                                      0.27  
Ex              No explanation found                                            NaN  
Fa              No explanat

In [63]:
start_time = time.time()
conj_res = multi_attribute_groupby.explain(explainer='many to one', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(houses_comparison_df, 'Multi attribute groupby', 'conjunctive', conj_res, conj_time)
conj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Conjunctive explanation runtime: 1.3080627918243408


,,Coverage,Separation Error
Group / Cluster,Explanation,,
"(4, 'TA')",No explanation found,NaN,NaN
"(5, 'Gd')",No explanation found,NaN,NaN
"(5, 'TA')",No explanation found,NaN,NaN
"(6, 'Gd')",No explanation found,NaN,NaN
"(6, 'TA')",No explanation found,NaN,NaN
"(7, 'Gd')",No explanation found,NaN,NaN
"(7, 'TA')",No explanation found,NaN,NaN
"(8, 'Gd')",No explanation found,NaN,NaN
"(9, 'Ex')",No explanation found,NaN,NaN


In [64]:
start_time = time.time()
disj_res = multi_attribute_groupby.explain(explainer='many to one', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(houses_comparison_df, 'Multi attribute groupby', 'disjunctive', disj_res, disj_time)
disj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Disjunctive explanation runtime: 1.1448500156402588


,,Coverage,Separation Error
Group / Cluster,Explanation,,
"(4, 'TA')",No explanation found,NaN,NaN
"(5, 'Gd')",No explanation found,NaN,NaN
"(5, 'TA')",No explanation found,NaN,NaN
"(6, 'Gd')",No explanation found,NaN,NaN
"(6, 'TA')",No explanation found,NaN,NaN
"(7, 'Gd')",No explanation found,NaN,NaN
"(7, 'TA')",No explanation found,NaN,NaN
"(8, 'Gd')",No explanation found,NaN,NaN
"(9, 'Ex')",No explanation found,NaN,NaN


### Clustering test

#### 3 clusters

In [65]:
labels = KMeans(n_clusters=3).fit_predict(pd.get_dummies(houses))

In [66]:
start_time = time.time()
conj_res = houses.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(houses_comparison_df, '3 clusters', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.796339750289917


Coverage Separation Error
Group / Cluster Explanation                                                
0               34900 <= SalePrice <= 173816.5         1.0              0.0
1               173816.5 <= SalePrice <= 755000        1.0             0.18
                178920.0 <= SalePrice <= 322940.0      0.9             0.07
2               296246.5 <= SalePrice <= 755000        1.0              0.0

In [67]:
start_time = time.time()
disj_res = houses.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(houses_comparison_df, '3 clusters', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.7269830703735352


Coverage Separation Error
Group / Cluster Explanation                                                
0               34900 <= SalePrice <= 173816.5         1.0              0.0
1               178920.0 <= SalePrice <= 322940.0      0.9             0.07
                173816.5 <= SalePrice <= 755000        1.0             0.18
2               296246.5 <= SalePrice <= 755000        1.0              0.0

#### 15 clusters

In [68]:
labels = KMeans(n_clusters=15).fit_predict(pd.get_dummies(houses))

In [69]:
start_time = time.time()
conj_res = houses.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(houses_comparison_df, '15 clusters', 'conjunctive', conj_res, conj_time)
conj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Conjunctive explanation runtime: 2.163193464279175


Coverage  \
Group / Cluster Explanation                                                   
0               128100.0 <= SalePrice <= 151200.0 AND 483.0 <= ...     0.78   
                128100.0 <= SalePrice <= 151200.0                      0.99   
3               103100.0 <= SalePrice <= 128100.0                       1.0   
5               301032.0 <= SalePrice <= 358165.0                      0.97   
7               72500 <= SalePrice <= 103100.0                          1.0   
9               150825.0 <= SalePrice <= 174250.0 AND Exterior1...     0.84   
                150825.0 <= SalePrice <= 174250.0                       1.0   
                150825.0 <= SalePrice <= 174250.0 AND 20.0 <= M...     0.84   
12              352000.0 <= SalePrice <= 415298                         1.0   
1               No explanation found                                    NaN   
4               No explanation found                                    NaN   
11              No explanation found                                    NaN   
14              No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               128100.0 <= SalePrice <= 151200.0 AND 483.0 <= ...              0.0  
                128100.0 <= SalePrice <= 151200.0                              0.01  
3               103100.0 <= SalePrice <= 128100.0                              0.01  
5               301032.0 <= SalePrice <= 358165.0                              0.02  
7               72500 <= SalePrice <= 103100.0                                  0.0  
9               150825.0 <= SalePrice <= 174250.0 AND Exterior1...              0.0  
                150825.0 <= SalePrice <= 174250.0                              0.01  
                150825.0 <= SalePrice <= 174250.0 AND 20.0 <= M...              0.0  
12              352000.0 <= SalePrice <= 415298                                 0.0  
1               No explanation found                                            NaN  
4               No explanation found                                            NaN  
11              No explanation found                                            NaN  
14              No explanation found                                            NaN

In [70]:
start_time = time.time()
disj_res = houses.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(houses_comparison_df, '15 clusters', 'disjunctive', disj_res, disj_time)
disj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Disjunctive explanation runtime: 1.5325193405151367


,,Coverage,Separation Error
Group / Cluster,Explanation,,
0,128100.0 <= SalePrice <= 151200.0,0.99,0.01
3,103100.0 <= SalePrice <= 128100.0,1.0,0.01
5,301032.0 <= SalePrice <= 358165.0,0.97,0.02
7,72500 <= SalePrice <= 103100.0,1.0,0.0
9,150825.0 <= SalePrice <= 174250.0,1.0,0.01
12,352000.0 <= SalePrice <= 415298,1.0,0.0
1,No explanation found,NaN,NaN
4,No explanation found,NaN,NaN
11,No explanation found,NaN,NaN


### Summary of results

In [71]:
conj_averages = houses_comparison_df.loc[:, 'conjunctive', :].mean()
disj_averages = houses_comparison_df.loc[:, 'disjunctive', :].mean()
houses_comparison_df.loc[('Average', 'conjunctive'), :] = conj_averages
houses_comparison_df.loc[('Average', 'disjunctive'), :] = disj_averages

In [72]:
houses_comparison_df

Average Coverage  \
Test                     Explanation Type                    
Full dataset             conjunctive              0.862727   
                         disjunctive              0.893125   
Numeric only             conjunctive              0.892778   
                         disjunctive                  0.93   
Categorical only         conjunctive                  0.82   
                         disjunctive               0.81625   
Single attribute groupby conjunctive              0.815238   
                         disjunctive              0.919524   
Multi attribute groupby  conjunctive                   NaN   
                         disjunctive                   NaN   
3 clusters               conjunctive                 0.975   
                         disjunctive                 0.975   
15 clusters              conjunctive              0.935556   
                         disjunctive              0.993333   
Average                  conjunctive               0.88355   
                         disjunctive              0.921205   

                                          Average Separation Error  \
Test                     Explanation Type                            
Full dataset             conjunctive                      0.045682   
                         disjunctive                      0.105625   
Numeric only             conjunctive                      0.130556   
                         disjunctive                      0.168571   
Categorical only         conjunctive                          0.03   
                         disjunctive                       0.05125   
Single attribute groupby conjunctive                      0.184048   
                         disjunctive                      0.209048   
Multi attribute groupby  conjunctive                           NaN   
                         disjunctive                           NaN   
3 clusters               conjunctive                        0.0625   
                         disjunctive                        0.0625   
15 clusters              conjunctive                      0.005556   
                         disjunctive                      0.008333   
Average                  conjunctive                       0.07639   
                         disjunctive                      0.100888   

                                          % Explanation Not Found  \
Test                     Explanation Type                           
Full dataset             conjunctive                         25.0   
                         disjunctive                         50.0   
Numeric only             conjunctive                         75.0   
                         disjunctive                         75.0   
Categorical only         conjunctive                         37.5   
                         disjunctive                         50.0   
Single attribute groupby conjunctive                         50.0   
                         disjunctive                         50.0   
Multi attribute groupby  conjunctive                        100.0   
                         disjunctive                        100.0   
3 clusters               conjunctive                            0   
                         disjunctive                            0   
15 clusters              conjunctive                         40.0   
                         disjunctive                         40.0   
Average                  conjunctive                    46.785714   
                         disjunctive                    52.142857   

                                          Average Explanation Length  \
Test                     Explanation Type                              
Full dataset             conjunctive                        1.613636   
                         disjunctive                          1.1875   
Numeric only             conjunctive                        1.611111   
                         disjunctive                         

## Bank churn dataset

In [73]:
bank_df = pd.read_csv(r"..\Examples\Datasets\bank_churners_user_study.csv")
bank_df = bank_df.drop(columns=['CLIENTNUM'])
bank_df

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Registered_Products_Count,Months_Inactive_Count_Last_Year,Contacts_Count_Last_Year,Credit_Limit,Credit_Used,Credit_Open_To_Buy,Total_Amount_Change_Q4_vs_Q1,Total_Transitions_Amount,Total_Transitions_Count,Total_Count_Change_Q4_vs_Q1,Credit_Avg_Utilization_Ratio
0,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,Existing Customer,50,M,2,Graduate,Single,$40K - $60K,Blue,40,3,2,3,4003.0,1851,2152.0,0.703,15476,117,0.857,0.462
10123,Attrited Customer,41,M,2,Unknown,Divorced,$40K - $60K,Blue,25,4,2,3,4277.0,2186,2091.0,0.804,8764,69,0.683,0.511
10124,Attrited Customer,44,F,1,High School,Married,Less than $40K,Blue,36,5,3,4,5409.0,0,5409.0,0.819,10291,60,0.818,0.000
10125,Attrited Customer,30,M,2,Graduate,Unknown,$40K - $60K,Blue,36,4,3,3,5281.0,0,5281.0,0.535,8395,62,0.722,0.000


In [74]:
bank_df_comparison_df = pd.DataFrame(columns=columns, index=index)

### Full dataset test

In [75]:
start_time = time.time()
conj_res = bank_df.explain(explainer='many to one', labels='Attrition_Flag', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(bank_df_comparison_df, 'Full dataset', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.6959044933319092


Coverage  \
Group / Cluster   Explanation                                       
Existing Customer 537.5 <= Credit_Used <= 2415.5             0.76   
                  54.5 <= Total_Transitions_Count <= 131     0.74   
Attrited Customer No explanation found                        NaN   

                                                         Separation Error  
Group / Cluster   Explanation                                              
Existing Customer 537.5 <= Credit_Used <= 2415.5                     0.07  
                  54.5 <= Total_Transitions_Count <= 131             0.05  
Attrited Customer No explanation found                                NaN

In [76]:
start_time = time.time()
disj_res = bank_df.explain(explainer='many to one', labels='Attrition_Flag', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(bank_df_comparison_df, 'Full dataset', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.7413156032562256


Coverage  \
Group / Cluster   Explanation                                       
Existing Customer 54.5 <= Total_Transitions_Count <= 131     0.74   
                  537.5 <= Credit_Used <= 2415.5             0.76   
Attrited Customer No explanation found                        NaN   

                                                         Separation Error  
Group / Cluster   Explanation                                              
Existing Customer 54.5 <= Total_Transitions_Count <= 131             0.05  
                  537.5 <= Credit_Used <= 2415.5                     0.07  
Attrited Customer No explanation found                                NaN

### Numeric only and categorical only tests

In [77]:
numeric_only = bank_df[['Customer_Age', 'Dependent_count', 'Months_on_book', 'Registered_Products_Count', 'Months_Inactive_Count_Last_Year', 'Contacts_Count_Last_Year', 'Credit_Limit', 'Credit_Used', 'Credit_Open_To_Buy', 'Total_Amount_Change_Q4_vs_Q1', 'Total_Transitions_Amount', 'Total_Transitions_Count', 'Total_Count_Change_Q4_vs_Q1', 'Credit_Avg_Utilization_Ratio']]
categorical_only = bank_df[['Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category']]
labels = bank_df['Attrition_Flag']

In [78]:
start_time = time.time()
conj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(bank_df_comparison_df, 'Numeric only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.6578261852264404


Coverage  \
Group / Cluster   Explanation                                       
Existing Customer 537.5 <= Credit_Used <= 2415.5             0.76   
                  54.5 <= Total_Transitions_Count <= 131     0.74   
Attrited Customer No explanation found                        NaN   

                                                         Separation Error  
Group / Cluster   Explanation                                              
Existing Customer 537.5 <= Credit_Used <= 2415.5                     0.07  
                  54.5 <= Total_Transitions_Count <= 131             0.05  
Attrited Customer No explanation found                                NaN

In [79]:
start_time = time.time()
disj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(bank_df_comparison_df, 'Numeric only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.7017171382904053


Coverage  \
Group / Cluster   Explanation                                       
Existing Customer 54.5 <= Total_Transitions_Count <= 131     0.74   
                  537.5 <= Credit_Used <= 2415.5             0.76   
Attrited Customer No explanation found                        NaN   

                                                         Separation Error  
Group / Cluster   Explanation                                              
Existing Customer 54.5 <= Total_Transitions_Count <= 131             0.05  
                  537.5 <= Credit_Used <= 2415.5                     0.07  
Attrited Customer No explanation found                                NaN

In [80]:
start_time = time.time()
conj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(bank_df_comparison_df, 'Categorical only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.14700865745544434


,,Coverage,Separation Error
Group / Cluster,Explanation,,
Existing Customer,Card_Category != Category_infrequent_sklearn,1.0,0.17
Attrited Customer,No explanation found,NaN,NaN


In [81]:
start_time = time.time()
disj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(bank_df_comparison_df, 'Categorical only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.08714985847473145


,,Coverage,Separation Error
Group / Cluster,Explanation,,
Existing Customer,Card_Category != Category_infrequent_sklearn,1.0,0.17
Attrited Customer,No explanation found,NaN,NaN


### Groupby tests

In [82]:
single_attribute_groupby = bank_df.groupby('Education_Level').mean()
multi_attribute_groupby = bank_df.groupby(['Education_Level', 'Income_Category']).mean()

In [83]:
start_time = time.time()
conj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(bank_df_comparison_df, 'Single attribute groupby', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 9.903491497039795


,,Coverage,Separation Error
Group / Cluster,Explanation,,
College,No explanation found,NaN,NaN
Doctorate,No explanation found,NaN,NaN
Graduate,No explanation found,NaN,NaN
High School,No explanation found,NaN,NaN
Post-Graduate,No explanation found,NaN,NaN
Uneducated,No explanation found,NaN,NaN
Unknown,No explanation found,NaN,NaN


In [84]:
start_time = time.time()
disj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(bank_df_comparison_df, 'Single attribute groupby', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 10.580077171325684


,,Coverage,Separation Error
Group / Cluster,Explanation,,
College,No explanation found,NaN,NaN
Doctorate,No explanation found,NaN,NaN
Graduate,No explanation found,NaN,NaN
High School,No explanation found,NaN,NaN
Post-Graduate,No explanation found,NaN,NaN
Uneducated,No explanation found,NaN,NaN
Unknown,No explanation found,NaN,NaN


In [85]:
start_time = time.time()
conj_res = multi_attribute_groupby.explain(explainer='many to one', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(bank_df_comparison_df, 'Multi attribute groupby', 'conjunctive', conj_res, conj_time)
conj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Conjunctive explanation runtime: 7.984471321105957


,,Coverage,Separation Error
Group / Cluster,Explanation,,
"('College', 'Less than $40K')",No explanation found,NaN,NaN
"('Graduate', '$40K - $60K')",No explanation found,NaN,NaN
"('Graduate', '$60K - $80K')",No explanation found,NaN,NaN
"('Graduate', '$80K - $120K')",No explanation found,NaN,NaN
"('Graduate', 'Less than $40K')",No explanation found,NaN,NaN
"('Graduate', 'Unknown')",No explanation found,NaN,NaN
"('High School', '$40K - $60K')",No explanation found,NaN,NaN
"('High School', 'Less than $40K')",No explanation found,NaN,NaN
"('Uneducated', 'Less than $40K')",No explanation found,NaN,NaN


In [86]:
start_time = time.time()
disj_res = multi_attribute_groupby.explain(explainer='many to one', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(bank_df_comparison_df, 'Multi attribute groupby', 'disjunctive', disj_res, disj_time)
disj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Disjunctive explanation runtime: 8.00534439086914


,,Coverage,Separation Error
Group / Cluster,Explanation,,
"('College', 'Less than $40K')",No explanation found,NaN,NaN
"('Graduate', '$40K - $60K')",No explanation found,NaN,NaN
"('Graduate', '$60K - $80K')",No explanation found,NaN,NaN
"('Graduate', '$80K - $120K')",No explanation found,NaN,NaN
"('Graduate', 'Less than $40K')",No explanation found,NaN,NaN
"('Graduate', 'Unknown')",No explanation found,NaN,NaN
"('High School', '$40K - $60K')",No explanation found,NaN,NaN
"('High School', 'Less than $40K')",No explanation found,NaN,NaN
"('Uneducated', 'Less than $40K')",No explanation found,NaN,NaN


### Clustering test

#### 3 clusters

In [87]:
labels = KMeans(n_clusters=3).fit_predict(pd.get_dummies(bank_df))

In [88]:
start_time = time.time()
conj_res = bank_df.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(bank_df_comparison_df, '3 clusters', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.983729362487793


Coverage  \
Group / Cluster Explanation                                                   
0               22381.0 <= Credit_Limit <= 34516.0                     0.99   
                23490.1 <= Credit_Limit <= 34516.0                     0.93   
                21008.0 <= Credit_Open_To_Buy <= 34516.0                1.0   
                22381.0 <= Credit_Limit <= 34516.0 AND 21008.0 ...     0.99   
1               9085.0 <= Credit_Limit <= 34516.0 AND 3.0 <= Cr...     0.79   
                8631.25 <= Credit_Open_To_Buy <= 17259.5               0.71   
                8650.5 <= Credit_Limit <= 34516.0 AND 3.0 <= Cr...     0.82   
                8159.0 <= Credit_Limit <= 34516.0 AND 3.0 <= Cr...     0.83   
2               1438.3 <= Credit_Limit <= 9707.724999999999             1.0   
                1438.3 <= Credit_Limit <= 9707.724999999999 AND...      1.0   
                1438.3 <= Credit_Limit <= 8281.5                       0.98   
                3.0 <= Credit_Open_To_Buy <= 8631.25                    1.0   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               22381.0 <= Credit_Limit <= 34516.0                             0.01  
                23490.1 <= Credit_Limit <= 34516.0                              0.0  
                21008.0 <= Credit_Open_To_Buy <= 34516.0                       0.02  
                22381.0 <= Credit_Limit <= 34516.0 AND 21008.0 ...              0.0  
1               9085.0 <= Credit_Limit <= 34516.0 AND 3.0 <= Cr...             0.01  
                8631.25 <= Credit_Open_To_Buy <= 17259.5                        0.0  
                8650.5 <= Credit_Limit <= 34516.0 AND 3.0 <= Cr...             0.04  
                8159.0 <= Credit_Limit <= 34516.0 AND 3.0 <= Cr...             0.11  
2               1438.3 <= Credit_Limit <= 9707.724999999999                    0.04  
                1438.3 <= Credit_Limit <= 9707.724999999999 AND...             0.02  
                1438.3 <= Credit_Limit <= 8281.5                                0.0  
                3.0 <= Credit_Open_To_Buy <= 8631.25                           0.04

In [89]:
start_time = time.time()
disj_res = bank_df.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(bank_df_comparison_df, '3 clusters', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.8672430515289307


Coverage  \
Group / Cluster Explanation                                                   
0               22381.0 <= Credit_Limit <= 34516.0                     0.99   
                23490.1 <= Credit_Limit <= 34516.0                     0.93   
                23490.1 <= Credit_Limit <= 34516.0 OR 23011.666...     0.94   
                21008.0 <= Credit_Open_To_Buy <= 34516.0                1.0   
1               8631.25 <= Credit_Open_To_Buy <= 17259.5               0.71   
2               1438.3 <= Credit_Limit <= 9707.724999999999             1.0   
                3.0 <= Credit_Open_To_Buy <= 6987.0 OR 1438.3 <...     0.99   
                1438.3 <= Credit_Limit <= 8281.5                       0.98   
                3.0 <= Credit_Open_To_Buy <= 8631.25                    1.0   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               22381.0 <= Credit_Limit <= 34516.0                             0.01  
                23490.1 <= Credit_Limit <= 34516.0                              0.0  
                23490.1 <= Credit_Limit <= 34516.0 OR 23011.666...              0.0  
                21008.0 <= Credit_Open_To_Buy <= 34516.0                       0.02  
1               8631.25 <= Credit_Open_To_Buy <= 17259.5                        0.0  
2               1438.3 <= Credit_Limit <= 9707.724999999999                    0.04  
                3.0 <= Credit_Open_To_Buy <= 6987.0 OR 1438.3 <...              0.0  
                1438.3 <= Credit_Limit <= 8281.5                                0.0  
                3.0 <= Credit_Open_To_Buy <= 8631.25                           0.04

#### 15 clusters

In [90]:
labels = KMeans(n_clusters=15).fit_predict(pd.get_dummies(bank_df))

In [91]:
start_time = time.time()
conj_res = bank_df.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(bank_df_comparison_df, '15 clusters', 'conjunctive', conj_res, conj_time)
conj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Conjunctive explanation runtime: 3.1712865829467773


Coverage  \
Group / Cluster Explanation                                                   
0               3399.0 <= Total_Transitions_Amount <= 6268.0 AN...     0.83   
                24.0 <= Credit_Open_To_Buy <= 2959.0 AND 3393.0...     0.98   
                24.0 <= Credit_Open_To_Buy <= 2959.0 AND 3399.0...     0.95   
                3306.5 <= Total_Transitions_Amount <= 17744 AND...     0.99   
                24.0 <= Credit_Open_To_Buy <= 5772.666666666667...      1.0   
...                                                                     ...   
14              24.0 <= Credit_Open_To_Buy <= 2959.0 AND 530.0 ...     0.99   
                530.0 <= Total_Transitions_Amount <= 3399.0 AND...      0.8   
                530.0 <= Total_Transitions_Amount <= 3399.0 AND...     0.99   
                24.0 <= Credit_Open_To_Buy <= 2959.0 AND 530.0 ...     0.99   
                24.0 <= Credit_Open_To_Buy <= 2959.0 AND 530 <=...     0.74   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               3399.0 <= Total_Transitions_Amount <= 6268.0 AN...              0.0  
                24.0 <= Credit_Open_To_Buy <= 2959.0 AND 3393.0...             0.06  
                24.0 <= Credit_Open_To_Buy <= 2959.0 AND 3399.0...             0.02  
                3306.5 <= Total_Transitions_Amount <= 17744 AND...             0.11  
                24.0 <= Credit_Open_To_Buy <= 5772.666666666667...              0.3  
...                                                                             ...  
14              24.0 <= Credit_Open_To_Buy <= 2959.0 AND 530.0 ...             0.08  
                530.0 <= Total_Transitions_Amount <= 3399.0 AND...             0.02  
                530.0 <= Total_Transitions_Amount <= 3399.0 AND...             0.08  
                24.0 <= Credit_Open_To_Buy <= 2959.0 AND 530.0 ...             0.03  
                24.0 <= Credit_Open_To_Buy <= 2959.0 AND 530 <=...             0.01  

[69 rows x 2 columns]

In [92]:
start_time = time.time()
disj_res = bank_df.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(bank_df_comparison_df, '15 clusters', 'disjunctive', disj_res, disj_time)
disj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Disjunctive explanation runtime: 1.8690433502197266


Coverage  \
Group / Cluster Explanation                                                   
2               12464.199999999999 <= Credit_Limit <= 17977.149...     0.88   
8               12006.0 <= Total_Transitions_Amount <= 17744.0          1.0   
                10922.0 <= Total_Transitions_Amount <= 17744            1.0   
9               25893.0 <= Credit_Open_To_Buy <= 34516.0                1.0   
                28767.333333333336 <= Credit_Open_To_Buy <= 345...      1.0   
                27466.5 <= Credit_Open_To_Buy <= 34516.0                1.0   
                27617.6 <= Credit_Open_To_Buy <= 34516.0                1.0   
11              21284.919999999995 <= Credit_Limit <= 27900.459...     0.93   
                20719.199999999997 <= Credit_Open_To_Buy <= 276...     0.81   
                21284.919999999995 <= Credit_Limit <= 27900.459...     0.96   
12              2959.0 <= Credit_Open_To_Buy <= 5830.0                  0.8   
0               No explanation found                                    NaN   
4               No explanation found                                    NaN   
5               No explanation found                                    NaN   
10              No explanation found                                    NaN   
14              No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
2               12464.199999999999 <= Credit_Limit <= 17977.149...             0.28  
8               12006.0 <= Total_Transitions_Amount <= 17744.0                  0.0  
                10922.0 <= Total_Transitions_Amount <= 17744                    0.0  
9               25893.0 <= Credit_Open_To_Buy <= 34516.0                        0.0  
                28767.333333333336 <= Credit_Open_To_Buy <= 345...              0.0  
                27466.5 <= Credit_Open_To_Buy <= 34516.0                        0.0  
                27617.6 <= Credit_Open_To_Buy <= 34516.0                        0.0  
11              21284.919999999995 <= Credit_Limit <= 27900.459...             0.01  
                20719.199999999997 <= Credit_Open_To_Buy <= 276...              0.0  
                21284.919999999995 <= Credit_Limit <= 27900.459...             0.01  
12              2959.0 <= Credit_Open_To_Buy <= 5830.0                          0.3  
0               No explanation found                                            NaN  
4               No explanation found                                            NaN  
5               No explanation found                                            NaN  
10              No explanation found                                            NaN  
14              No explanation found                                            NaN

### Summary of results

In [93]:
conj_averages = bank_df_comparison_df.loc[:, 'conjunctive', :].mean()
disj_averages = bank_df_comparison_df.loc[:, 'disjunctive', :].mean()
bank_df_comparison_df.loc[('Average', 'conjunctive'), :] = conj_averages
bank_df_comparison_df.loc[('Average', 'disjunctive'), :] = disj_averages

In [94]:
bank_df_comparison_df

Average Coverage  \
Test                     Explanation Type                    
Full dataset             conjunctive                  0.75   
                         disjunctive                  0.75   
Numeric only             conjunctive                  0.75   
                         disjunctive                  0.75   
Categorical only         conjunctive                   1.0   
                         disjunctive                   1.0   
Single attribute groupby conjunctive                   NaN   
                         disjunctive                   NaN   
Multi attribute groupby  conjunctive                   NaN   
                         disjunctive                   NaN   
3 clusters               conjunctive                  0.92   
                         disjunctive              0.948889   
15 clusters              conjunctive              0.874493   
                         disjunctive              0.943636   
Average                  conjunctive              0.858899   
                         disjunctive              0.878505   

                                          Average Separation Error  \
Test                     Explanation Type                            
Full dataset             conjunctive                          0.06   
                         disjunctive                          0.06   
Numeric only             conjunctive                          0.06   
                         disjunctive                          0.06   
Categorical only         conjunctive                          0.17   
                         disjunctive                          0.17   
Single attribute groupby conjunctive                           NaN   
                         disjunctive                           NaN   
Multi attribute groupby  conjunctive                           NaN   
                         disjunctive                           NaN   
3 clusters               conjunctive                      0.024167   
                         disjunctive                      0.012222   
15 clusters              conjunctive                      0.136812   
                         disjunctive                      0.054545   
Average                  conjunctive                      0.090196   
                         disjunctive                      0.071354   

                                          % Explanation Not Found  \
Test                     Explanation Type                           
Full dataset             conjunctive                         50.0   
                         disjunctive                         50.0   
Numeric only             conjunctive                         50.0   
                         disjunctive                         50.0   
Categorical only         conjunctive                         50.0   
                         disjunctive                         50.0   
Single attribute groupby conjunctive                        100.0   
                         disjunctive                        100.0   
Multi attribute groupby  conjunctive                        100.0   
                         disjunctive                        100.0   
3 clusters               conjunctive                            0   
                         disjunctive                            0   
15 clusters              conjunctive                            0   
                         disjunctive                         50.0   
Average                  conjunctive                         50.0   
                         disjunctive                    57.142857   

                                          Average Explanation Length  \
Test                     Explanation Type                              
Full dataset             conjunctive                             1.0   
                         disjunctive                             1.0   
Numeric only             conjunctive                             1.0   
                         disjunctive                         

## Summary of all results

In [95]:
results_summary = pd.DataFrame(columns=columns, index=pd.MultiIndex(levels=[[], [], []], codes=[[], [], []], names=['Dataset', 'Test', 'Explanation Type']))

In [96]:
def build_final_df(final_df, dataset_df, dataset_name):
    for row in dataset_df.iterrows():
        final_df.loc[(dataset_name, *row[0]), :] = row[1]

In [97]:
build_final_df(results_summary, adults_comparison_df, 'Adults')
build_final_df(results_summary, spotify_comparison_df, 'Spotify')
build_final_df(results_summary, houses_comparison_df, 'Houses')
build_final_df(results_summary, bank_df_comparison_df, 'Bank churn')

In [98]:
# Compute the average across all datasets, without the 'average' row for each dataset
all_datasets_avg_conj_averages = results_summary.loc[(slice(None), slice(None), 'conjunctive'), :].drop('Average', level=1).mean()
all_datasets_avg_disj_averages = results_summary.loc[(slice(None), slice(None), 'disjunctive'), :].drop('Average', level=1).mean()
results_summary.loc[('All Datasets', 'Average', 'conjunctive'), :] = all_datasets_avg_conj_averages
results_summary.loc[('All Datasets', 'Average', 'disjunctive'), :] = all_datasets_avg_disj_averages

In [99]:
results_summary

Average Coverage  \
Dataset      Test             Explanation Type                    
Adults       Full dataset     conjunctive                0.8725   
                              disjunctive              0.913333   
             Numeric only     conjunctive                0.8725   
                              disjunctive              0.913333   
             Categorical only conjunctive                  0.88   
...                                                         ...   
Bank churn   15 clusters      disjunctive              0.943636   
             Average          conjunctive              0.858899   
                              disjunctive              0.878505   
All Datasets Average          conjunctive              0.902117   
                              disjunctive              0.936511   

                                               Average Separation Error  \
Dataset      Test             Explanation Type                            
Adults       Full dataset     conjunctive                        0.1725   
                              disjunctive                      0.186667   
             Numeric only     conjunctive                        0.1725   
                              disjunctive                      0.186667   
             Categorical only conjunctive                      0.196667   
...                                                                 ...   
Bank churn   15 clusters      disjunctive                      0.054545   
             Average          conjunctive                      0.090196   
                              disjunctive                      0.071354   
All Datasets Average          conjunctive                      0.094948   
                              disjunctive                      0.107581   

                                               % Explanation Not Found  \
Dataset      Test             Explanation Type                           
Adults       Full dataset     conjunctive                         50.0   
                              disjunctive                         50.0   
             Numeric only     conjunctive                         50.0   
                              disjunctive                         50.0   
             Categorical only conjunctive                         50.0   
...                                                                ...   
Bank churn   15 clusters      disjunctive                         50.0   
             Average          conjunctive                         50.0   
                              disjunctive                    57.142857   
All Datasets Average          conjunctive                    45.709877   
                              disjunctive                    51.419753   

                                               Average Explanation Length  \
Dataset      Test             Explanation Type                              
Adults       Full dataset     conjunctive                            1.25   
                              disjunctive                             1.0   
             Numeric only     conjunctive                            1.25   
                              disjunctive                             1.0   
             Categorical only conjunctive                        1.333333   
...                                                                   ...   
Bank churn   15 clusters      disjunctive                        1.090909   
             Average          conjunctive                        1.335507   
                              disjunctive                        1.062626   
All Datasets Average          conjunctive                        1.336802   
                              disjunctive                        1.114775   

                                               Number of Explanations Found  \
Dataset      Test             Explanation Type                                
Adults       Full dataset     conjunctive                                 4 